# Join the data downloaded from Entso-e to create a unified DataFrame

Create one datframe for day-ahead and one for week-ahead

In [20]:
import pandas as pd
import datetime
import os

from Name_convention_dictionaries import PsrTypeDict

In [21]:
# make sure the data type of the dates is datetime:
dateparse = lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

## Join all the files for day-ahead

In [22]:
# get a list of all the day-ahead files
folder_name = 'data_day_ahead'
files_in_dir = os.listdir("./"+folder_name+"/")

Goal is to create a DataFrame for each document and process type. This means, all the data from regarding e.g. total load is saved in one big DataFrame:

In [23]:
check_if_loaded = []
list_of_dfs = []

# take the first file in the folder
for i in files_in_dir:
    group_files = []
    
    # check if the file has been used before
    if i == '.DS_Store':
        pass
    
    elif i[:-38] not in check_if_loaded:
        
        # you want to group all files in the folder that belong to the same document and process type
        # search for all files that have the same characters at the beginning of the file name
        for j in files_in_dir:
            
            # if the selected file has the same characters at the beginning of the file name, save the
            # path in the group_files list
            if i[:-38] == j[:-38]:
                group_files.append("./"+folder_name+"/"+j)
        
        # save the first file in the group_files list as a dataframe in memory
        df = pd.read_csv(group_files[0],parse_dates=['Date'], date_parser=dateparse)
        
        # iterate through all the remaining files in the group_files list and store them as a dataframe
        # in memory. Then concatenate all dataframes
        for k in group_files[1:]:
            df2 = pd.read_csv(k,parse_dates=['Date'], date_parser=dateparse)
            df = pd.concat([df,df2])

        # Finally, sort the values in the dataframe by datetime and append the dataframe to the
        # list_of_dfs. This will be used later to merge all dataframes together
        df = df.sort_values(by=["Date"])
        list_of_dfs.append(df)
        
        # Append the first file to the check_if_loaded list to make sure all files with the same document
        # and process type are ignored for the next iteration in the loop
        check_if_loaded.append(i[:-38])

In [24]:
# search for duplicates and get rid of them
for i in list_of_dfs:
    print(i.duplicated().sum())
    i.drop_duplicates(inplace=True)
    print(i.duplicated().sum())

64328
0
203397
0
64232
0
64232
0
64232
0
203553
0
203993
0
203564
0


## Merge all DataFrames

In [25]:
from functools import reduce

In [26]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='outer'), list_of_dfs)

In [27]:
df_merged = df_merged.sort_values(by=["Date"]).reset_index()

In [28]:
# save the new dataframe as a csv file
df_merged.to_csv("Day_ahead_dataset.csv", index=False)

In [29]:
# make sure all columns have been stored
df_merged.columns

Index(['index', 'Date', 'Day ahead/System total load in MAW',
       'Realised/Solar in MAW', 'Day ahead/Solar in MAW',
       'Day ahead/Wind Onshore in MAW', 'Day ahead/Wind Offshore in MAW',
       'Realised/Wind Onshore in MAW', 'Realised/System total load in MAW',
       'Realised/Wind Offshore in MAW'],
      dtype='object')

In [30]:
# check how many nan values are in the DataFrame
df_merged.isnull().sum()

index                                   0
Date                                    0
Day ahead/System total load in MAW     96
Realised/Solar in MAW                 696
Day ahead/Solar in MAW                576
Day ahead/Wind Onshore in MAW         576
Day ahead/Wind Offshore in MAW         96
Realised/Wind Onshore in MAW          540
Realised/System total load in MAW     100
Realised/Wind Offshore in MAW         529
dtype: int64

In [31]:
df_merged.sample(10)

,index,Date,Day ahead/System total load in MAW,Realised/Solar in MAW,Day ahead/Solar in MAW,Day ahead/Wind Onshore in MAW,Day ahead/Wind Offshore in MAW,Realised/Wind Onshore in MAW,Realised/System total load in MAW,Realised/Wind Offshore in MAW
62301,62301,2016-10-10 22:15:00,45588.0,0.0,0.0,2863.0,1853.0,1979.0,47864.0,1497.0
186770,186674,2020-04-29 11:30:00,62651.0,14292.0,15520.0,10895.0,1377.0,8660.0,61022.0,1278.0
46378,46378,2016-04-28 01:30:00,43617.0,0.0,0.0,9419.0,1118.0,9426.0,46538.0,1365.0
47031,47031,2016-05-04 20:45:00,49049.0,0.0,0.0,2974.0,444.0,3909.0,50779.0,295.0
45736,45736,2016-04-21 09:00:00,65438.0,23074.0,22947.0,2795.0,551.0,1785.0,65970.0,534.0
56033,56033,2016-08-06 15:15:00,48465.0,9341.0,10056.0,9748.0,2014.0,8939.0,48809.0,1355.0
158751,158655,2019-07-12 14:45:00,60914.0,10445.0,11123.0,3631.0,326.0,5661.0,61855.0,640.0
180322,180226,2020-02-22 07:30:00,57256.0,5082.0,4268.0,35242.0,5165.0,35985.0,55519.0,3233.0
33182,33182,2015-12-12 14:30:00,56959.0,509.0,599.0,11043.0,1625.0,8413.0,55516.0,1422.0
180163,180067,2020-02-20 15:45:00,68169.0,1429.0,1096.0,25817.0,5897.0,28153.0,67182.0,5696.0


## Join all the files for week-ahead predictions

In [32]:
# get a list of all the day-ahead files
folder_name = 'data_week_ahead'
files_in_dir = os.listdir("./"+folder_name+"/")

The task for the week-ahead prediction is much simpler than day-ahead (as there are no actuals or generation values). use a simpler implementation to save some memory:

In [33]:
# create a list of the paths to the files. So they can be saved as DataFrames
list_of_paths = []
for name in files_in_dir:
    list_of_paths.append("./"+folder_name+"/"+name)

# store the DataFrames in memory and concatenate them
df_week = pd.read_csv(list_of_paths[0], parse_dates=['min_date', 'max_date'], date_parser=dateparse)
for file in list_of_paths[1:]:
    df2 = pd.read_csv(file, parse_dates=['min_date', 'max_date'], date_parser=dateparse)
    df_week = pd.concat([df_week, df2])
    
df_week.sort_values(by=['min_date', 'max_date'], inplace=True)

In [34]:
# get rid of all duplicates before saving the data as a csv
df_week.drop_duplicates(inplace=True)

df_week.to_csv('Week_ahead_dataset.csv', index=False)

In [35]:
df_week.sample(10)

,min_date,max_date,min_forecast_in_MAW,max_forecast_in_MAW
301,2016-09-12 22:00:00,2016-09-12 22:00:00,37577,64954
81,2020-01-17 23:00:00,2020-01-17 23:00:00,48367,67684
94,2019-01-24 23:00:00,2019-01-24 23:00:00,52284,72776
257,2015-10-24 22:00:00,2015-10-24 22:00:00,40549,57156
140,2018-03-12 23:00:00,2018-03-12 23:00:00,45686,70838
110,2019-02-09 23:00:00,2019-02-09 23:00:00,48509,62658
20,2018-11-11 23:00:00,2018-11-11 23:00:00,40863,56475
29,2019-11-19 23:00:00,2019-11-19 23:00:00,48264,70047
283,2019-08-08 22:00:00,2019-08-08 22:00:00,40759,63715
137,2018-03-09 23:00:00,2018-03-09 23:00:00,50584,71165
